In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__, external_stylesheets=["https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css"])


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# pre-computing and sorting for filter menu
animal_types = sorted(df['animal_type'].dropna().unique())
breeds = sorted(df['breed'].dropna().unique())
outcomes = sorted(df['outcome_type'].dropna().unique())


app.layout = html.Div(className='container', style={'overflowX': 'hidden'}, children=[
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
        html.P("Dashboard created by Kemal Cankurt", style={'fontStyle': 'italic', 'marginTop': '10px'})
    ]),
    html.Hr(),
    html.Div(
    ),
    
    html.Div(className="card mb-4", children=[
    html.Div(className="card-body", children=[
    html.H5("Rescue Type", className="card-title"),
            dcc.RadioItems(
                id='filter-rescue-type',
                options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}],
                value='reset',
                labelStyle={'display': 'block'},
                inputStyle={'margin-right': '10px'}
            )
        
        ])
    ]),
    
    html.Div(className="card mb-4", children=[
    html.Div(className="card-body", children=[
        html.Div(className='row g-3', children=[
            html.Div(className='col-md-3', children=[
                html.Label("Animal Type"),
                dcc.Dropdown(
                    id='filter-animal-type',
                    options=[{'label': a, 'value': a} for a in animal_types],
                    placeholder="Select Animal Type"
                )
            ]),
            html.Div(className='col-md-3', children=[
                html.Label("Breed"),
                dcc.Dropdown(
                    id='filter-breed',
                    options=[{'label': b, 'value': b} for b in breeds],
                    placeholder="Select Breed"
                )
            ]),
            html.Div(className='col-md-3', children=[
                html.Label("Outcome Type"),
                dcc.Dropdown(
                    id='filter-outcome-type',
                    options=[{'label': o, 'value': o} for o in outcomes],
                    placeholder="Select Outcome"
                )
            ]),
            html.Div(className='col-md-3', children=[
                html.Label("Search Name"),
                dcc.Input(
                    id='filter-name',
                    type='text',
                    debounce=True,
                    placeholder="Enter animal name...",
                    className="form-control"
                )
            ])
        ])
    ])
]),


    html.Hr(),
    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 

    html.Div(className='row', children=[
        html.Div(className='col s12 m12', children=[
            dash_table.DataTable(
                id='datatable-id',
                columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                data=df.to_dict('records'),
                page_size=10,
                style_table={'max-height': '500px', 'overflowY': 'auto', 'overflowX': 'auto'},
                row_selectable='single',
                selected_rows=[0],
                sort_action='native',
                style_cell={
                    'textAlign': 'left',
                    'minWidth': '100px',
                    'width': '150px',
                    'maxWidth': '200px'
                },
                style_data={'whiteSpace': 'normal', 'height': 'auto'}
            )
        ])
    ]),

    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'height': '500px'}
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'height': '500px'}
        )
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'data'),
    [
        Input('filter-animal-type', 'value'),
        Input('filter-breed', 'value'),
        Input('filter-outcome-type', 'value'),
        Input('filter-name', 'value'),
        Input('filter-rescue-type', 'value')
    ]
)
def update_dashboard(animal_type, breed, outcome_type, name,rescue_type):
    query = {}
    
    if rescue_type == 'water':
        query.setdefault('animal_type', 'Dog')
        query.setdefault('breed', {'$in': ['Labrador Retriever Mix']})
    elif rescue_type == 'mountain':
        query.setdefault('animal_type', 'Dog')
        query.setdefault('breed', {'$in': [
        'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
        'Siberian Husky', 'Rottweiler']})
    elif rescue_type == 'disaster':
        query.setdefault('animal_type', 'Dog')
        query.setdefault('breed', {'$in': [
        'Doberman Pinscher', 'German Shepherd', 'Golden Retriever',
        'Bloodhound', 'Rottweiler']})
    
    if animal_type:
        query['animal_type'] = animal_type
    if breed:
    # If breed filter is already an $in list (from rescue_type), we apply AND logic
        if isinstance(query.get('breed'), dict) and '$in' in query['breed']:
            query['breed']['$in'] = [b for b in query['breed']['$in'] if b == breed]
        else:
            query['breed'] = breed
    if outcome_type:
        query['outcome_type'] = outcome_type
    if name:
            query['name'] = {'$regex': name, '$options': 'i'}

    results = db.read(query)
    df_filtered = pd.DataFrame(results)

    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty or 'breed' not in dff.columns:
        return [html.P("No data to display.")]

    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return [html.P("No location to display.")]

    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    lat = dff.iloc[row, 13]
    lon = dff.iloc[row, 14]

    return [
        dl.Map(style={'height': '450px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                      children=[
                          dl.Tooltip(dff.iloc[row,4]),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row,9])
                          ])
                      ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:16756/
